## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key


# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
# city_data_df = pd.read_csv("WeatherPy_vacation.csv")
# city_data_df.head()

city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

City_ID          City Country      Lat       Lng  Max Temp  Humidity  \
0        0       Ushuaia      AR -54.8000  -68.3000     38.86        81   
1        1  Ponta Do Sol      PT  32.6667  -17.1000     72.23        77   
2        2      La Ronge      CA  55.1001 -105.2842     82.44        25   
3        3      Yinchuan      CN  38.4681  106.2731     73.71        52   
4        4     Busselton      AU -33.6500  115.3333     54.19        75   

   Cloudiness  Wind Speed   Current Description  
0          75       21.85  heavy intensity rain  
1           9        7.20             clear sky  
2          95        3.96       overcast clouds  
3         100        2.39            light rain  
4         100        7.36       overcast clouds

#### Input statements are written to prompt the customer for their minimum and maximum temperature preferences

In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


#### A new DataFrame is created based on the minimum and maximum temperature, and empty rows are dropped (below). 

In [7]:
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

City_ID          City Country      Lat       Lng  Max Temp  Humidity  \
1         1  Ponta Do Sol      PT  32.6667  -17.1000     72.23        77   
2         2      La Ronge      CA  55.1001 -105.2842     82.44        25   
3         3      Yinchuan      CN  38.4681  106.2731     73.71        52   
5         5       Kavieng      PG  -2.5744  150.7967     83.48        76   
6         6         Mecca      SA  21.4267   39.8261     85.64        59   
10       10       Wanning      CN  18.8003  110.3967     80.78        85   
13       13       Prainha      BR  -1.8000  -53.4800     77.90        66   
14       14        Kawalu      ID  -7.3817  108.2082     71.49        98   
15       15       Changji      CN  44.0167   87.3167     74.19        25   
20       20        Atuona      PF  -9.8000 -139.0333     77.67        76   

    Cloudiness  Wind Speed Current Description  
1            9        7.20           clear sky  
2           95        3.96     overcast clouds  
3          100        2.39          light rain  
5           27       10.87          light rain  
6           67        8.61       broken clouds  
10          90        9.28     overcast clouds  
13          59        3.33       broken clouds  
14          98        2.06     overcast clouds  
15           0       11.18           clear sky  
20          18       19.69          few clouds

In [8]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID                293
City                   293
Country                293
Lat                    293
Lng                    293
Max Temp               293
Humidity               293
Cloudiness             293
Wind Speed             293
Current Description    293
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df.dropna(inplace = True) 

C:\Users\lindgp1\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### The hotel name is retrieved and added to the DataFrame, and the rows that don’t have a hotel name are dropped.

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

City Country  Max Temp Current Description      Lat       Lng  \
1   Ponta Do Sol      PT     72.23           clear sky  32.6667  -17.1000   
2       La Ronge      CA     82.44     overcast clouds  55.1001 -105.2842   
3       Yinchuan      CN     73.71          light rain  38.4681  106.2731   
5        Kavieng      PG     83.48          light rain  -2.5744  150.7967   
6          Mecca      SA     85.64       broken clouds  21.4267   39.8261   
10       Wanning      CN     80.78     overcast clouds  18.8003  110.3967   
13       Prainha      BR     77.90       broken clouds  -1.8000  -53.4800   
14        Kawalu      ID     71.49     overcast clouds  -7.3817  108.2082   
15       Changji      CN     74.19           clear sky  44.0167   87.3167   
20        Atuona      PF     77.67          few clouds  -9.8000 -139.0333   

   Hotel Name  
1              
2              
3              
5              
6              
10             
13             
14             
15             
20

In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.   
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json() 
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
hotel_df

City Country  Max Temp Current Description      Lat       Lng  \
1    Ponta Do Sol      PT     72.23           clear sky  32.6667  -17.1000   
2        La Ronge      CA     82.44     overcast clouds  55.1001 -105.2842   
3        Yinchuan      CN     73.71          light rain  38.4681  106.2731   
5         Kavieng      PG     83.48          light rain  -2.5744  150.7967   
6           Mecca      SA     85.64       broken clouds  21.4267   39.8261   
..            ...     ...       ...                 ...      ...       ...   
678        Karnal      IN     79.54       broken clouds  29.6833   76.9833   
679      West Bay      KY     89.08          few clouds  19.3667  -81.4167   
680         Arani      IN     81.45     overcast clouds  12.6667   79.2833   
681        Tiarei      PF     82.42     overcast clouds -17.5333 -149.3333   
683       Mindelo      CV     75.36       broken clouds  16.8901  -24.9804   

                                     Hotel Name  
1                                Hotel do Campo  
2                         Northland Motor Hotel  
3     JJ Inns - Yinchuan New Moon Square Branch  
5                           Nusa Island Retreat  
6    Makkah Clock Royal Tower, A Fairmont Hotel  
..                                          ...  
678                             Hotel Deventure  
679                                Calypso Cove  
680                                   KPK LODGE  
681                         Le Rocher de Tahiti  
683                                 Hotel Gaudi  

[293 rows x 7 columns]

In [23]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(inplace = True) 
len(hotel_df)

293

#### The DataFrame is exported as a CSV file into the Vacation_Search folder. Again I added an index label

In [24]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

#### Create pop up info - Current weather and max temp on one line

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

#### A marker layer map with pop-up markers for the cities in the vacation DataFrame is created.

In [21]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
# max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
#              max_intensity=300, point_radius=4)

# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))